# Recursion (General)

Recursion refers to self-referential algorithms. Any algorithm that can be solved iteratively can be solved recursively and vice versa (though sometimes the iterative implementation requires a stack, as recursion implicitly invokes the call stack). Some problems are easier to implement with one or the other; go with whatever feels most natural. 

When considering a recursive solution, you must identify:
- A recurrence relationship in the problem. It helps to write this out explicitly.
- A base case; this is a case where no self-reference is needed to provide a result.
- A recursive case; the cases where self reference is required. 

With this, you can then begin implementing a recursive function. Here's a simple template:
```
def recurse(value):
    if base case:
        return base value
    else:
        return recurse(next value) + recurse(other next value)

def handler(input):
    return recurse(first value)
```

If your language allows it, you can implement the recursive function inside the handler function. I find that this is helpful if the recursive function needs to access a global data structure, but it can complicate testing:
```
def handler(input):
    def recurse(value):
        if base case:
            return base value
        else:
            return recurse(next value) + recurse(other next value)
return recurse(first value)
```

Some common gotchas:
- Not identifying the recurrence relationship completely before implementing the recursive function will cause confusion and possibly lead you to miss edge cases. 
- Every recursive call must eventually hit the base case; otherwise you will recurse forever. 
- A recursive function usually should return a concrete value in the base case or a combination of return values for recursive cases. A common mistake is to forget to return a recursive call's return value.

## Example: Tower of Hannoi 

The [Tower of Hannoi](https://en.wikipedia.org/wiki/Tower_of_Hanoi) problem is a classical example of a problem with a recursive solution. We can solve it recursively by thinking about "subproblems" - the 4-high tower (assuming plates 1,2,3,4) is just a 3-high tower sitting on top of a 4-plate. If we have some technique to move a 3-high tower to an arbitrary peg, we just move the 3-high tower to the temporary peg, move the 4 plate to the destination peg, then move the 3-high tower on top of it. How would we move a 3-high tower? Well, if we had a way to move a 2-high tower....

The base case here is moving a 1-high tower (a zero-high tower involves no movement); we move it from source to destination. Note here that as long as we start with a valid tower (e.g. 4,3,2,1 with 1 being the top), we don't have to worry about intermediate moves producing illegal states - if while solving our 4-tower we solve a 3-tower, and while solving our 3-tower we solve a 2-tower, we still at every step only need to move smaller plates on top of larger plates. If we follow our algorithm correctly, we should always have a temporary and destination slot available (convince yourself with [this SVG](https://upload.wikimedia.org/wikipedia/commons/2/20/Tower_of_Hanoi_recursion_SMIL.svg); click the grey boxes to expand).


In [20]:
towers = [
    [4,3,2,1],
    [],
    []
]

def move_plate(source, dest, spaces):
    plate = towers[source].pop()
    towers[dest].append(plate)


def solve(largest_plate, source, dest, extra, spaces=0):
    if largest_plate == 1:
        # Base case; only one plate to move
        move_plate(source, dest, spaces)
    else:
        # Move the entire pile except the largest one to the extra space
        solve(largest_plate-1, source, extra, dest, spaces+2)
        # Move the bottom plate to the destination
        move_plate(source, dest, spaces)
        # Then move the entire pile on top of the largest plate at the destination
        solve(largest_plate-1, extra, dest, source, spaces+2)

print(towers)
solve(4,0,1,2)
print(towers)



[[4, 3, 2, 1], [], []]
[[], [4, 3, 2, 1], []]


## [17. Letter Combinations of a Phone Number](https://leetcode.com/problems/letter-combinations-of-a-phone-number/)

</br>
<details>
<summary><b>Click for answer.</b></summary>
Recurrence: To get all strings for the `i`th number in the input string, combine all of the `i`th numbers digits with all strings associated with the `i-1`th number. 

Suppose we have the following functions:
- `get_chars(i)` - returns characters associated with digit, e.g. `get_chars(2) -> ['a','b','c']`
- `combine(c, strs)` - given a char `c` and list of strings `strs`, returns a list containing every string in `strs` with `c` appended.  
- `get_combinations(s, i)` - a function that solves the problem, i.e. returns all letter combinations for an int string `s` of length `i`.

Base case: A single number `i`; return `get_chars(i)`
Recursive case: the `i`th number of our input string. `get_combinations(s, i) = combine(get_combinations(s, i-1), c) for c in get_chars(s[i])`

I actually got this as an interview problem once (during a period of my life when I was incompetent at interviewing) and promptly failed it after attempting a solution on the whiteboard with multiple layers of nested loops. Despite the experience leaving me a scarred and broken man (most medical experts agree that whiteboards are a worldwide leading cause of early death), I somehow got an offer and the interviewer later became my manager.
</details>

In [3]:
%run ../leetcode/17-letter-combinations-of-a-phone-number/solution.2.py

## [776. Split BST](https://leetcode.com/problems/split-bst/)
</br>
<details>
<summary><b>Click for answer.</b></summary>

An `O(N)` solution is possible and poorly explained on Leetcode. For a given tree T, we return `left` (every node in T where node.val <= V) and `right` (every node in T where node.val > V). Our base case is a single node - it either winds up in a left or right tree of one node, and the other tree is empty. For more than one node, we look at `root`. If `root` goes into `left`, we don't need to look at `root.left` since every value in the root's left subtree is also less than V, so we go on to look at `root.right`. Opposite logic applies if `root` goes into `right`; we go on to look at `root.left`. 

Let's say `V= 57` and we have this tree:
```
    0
   / \ 
-100  100
 / \  / \
   ....
```
We don't need to examine `root.left` since every value in it is less than 0, so we recurse into `root.right`. We then have three trees: the original `root` at `0`, and the `left` and `right` on our recursive call `split(root.right)`. In this case, we could merge them by setting `root.right = left`; every value in `root`'s original right subtree was greater than `root`, so this preserves the BST property. Then the recursive call's right is `right` and the original root is `left`, and we return both. Again, opposite logic applies if we examine `root.left` in the first case. I won't bother stating the recurrence here since it's more confusing than it's worth.
    
An alternative, brute-force `O(nlogn)` solution is to pre-order traverse the input tree and insert every node into the new left or right subtree. 
</details>


In [7]:
# No tests; passes LeetCode
%run ../leetcode/776-split-bst/solution.py
%run ../leetcode/776-split-bst/solution-nlogn.py

## [21. Merge Two Sorted Lists](https://leetcode.com/problems/merge-two-sorted-lists/)
</br>
<details>
<summary><b>Click for answer.</b></summary>
For this one, we can recursively use a mergesort-style merge. 
</details>

In [13]:
# No tests; passes LeetCode
%run ../leetcode/21-merge-two-sorted-lists/solution.2.py

## [24. Swap Nodes in Pairs](https://leetcode.com/problems/swap-nodes-in-pairs/)
</br>
<details>
<summary><b>Click for answer.</b></summary>
For this one, our recurrence is: `swapped list of N nodes = 2nd node + 1st node + swapped list of N-2 nodes`, with base cases for zero and one node.
</details>

In [14]:
%run ../leetcode/24-swap-nodes-in-pairs/solution.py

## [93. Restore IP Addresses](https://leetcode.com/problems/restore-ip-addresses/submissions/)
</br>
<details>
<summary><b>Click for answer.</b></summary>
  
This problem is an exhaustive search - we need to find all possible IP addresses from a given string of ints. Here we solve it with general recursion, but it can also be solved similarly with backtracking.

### Identifying a recurrence
A valid IPv4 address is of the form `x.x.x.x` where `0 <= x 255`, and x has no leading `0` (unless `x == 0`). Let's say that we have three types of IPs:
- A valid IP address, e.g. `24.123.54.4`
- An invalid IP address:
    - `525.1.2.4` (field exceeds 255)
    - `01.1.24.5` (field has leading zero)
    - `1..3.4` (empty field)
    - `1.2.3.4.5` (too many fields)
- An incomplete IP address, e.g. an IP that is only invalid because it has too few fields:
    - `1.2.3` 
    - `1.2`
    - `1.`
    - `(empty string)`

For any IP address `s` and a character `x` s.t. `0 <= x <= 9`, we can either try adding `x` as the final character of s, or appending `x` to the end of `s` as a new field:
- If `s` is valid:
    - We cannot append `x` as a new field, e.g. if `s = 1.2.3.4`, `1.2.3.4.x` is not valid. 
    - We can possibly appending `x` as a final character: `1.2.3.4x` is valid.
- If `s` is invalid, there is no way to append `x` to produce a valid or incomplete IP. 
- If `s` is incomplete:
    - Appending `x` as a field produces either a valid IP (`s = 1.2.3, x = 4, s.x = 1.2.3.4`) or another incomplete IP (`s = 1.2, x = 3, s.x = 1.2.3`)
    - Appending `x` as a final character produces an incomplete IP (`s = 1.2.1, x = 4, sx = 1.2.14`) or an invalid IP (`s = 1.2.99, x = 1, sx = 1.2.991`). 
    
So given an input `string` of int characters, our recurrence relationship is:
```
all valid IPs ending at string[i] =
    any valid result of appending s[i] as a character to all valid IPs ending at string[i-1],    
    any valid result of appending s[i] as a character to all incomplete IPs ending at string[i-1],
    any valid result of appending s[i] as a field to all incomplete IPs ending at string[i-1]
```

## Base case and recursive case
Because the problem says to use all characters in `string`, our base case will be: `current IP address is invalid, or current IP address is valid and i == len(string)`. The recursive case is `current IP is incomplete or i < len(string)`, and tries `append string[i] as character, go to i+1` and `append string[i] as a field, go to i+1)`.
</details>

In [1]:
%run ../leetcode/93-restore-ip-addresses/solution.2.py